In [32]:
import os
from langchain import PromptTemplate
import openai
import pinecone
from fastapi.middleware.cors import CORSMiddleware
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversational_retrieval.prompts import QA_PROMPT
from dotenv import load_dotenv, find_dotenv
from langchain.llms import AzureOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA

#check .env
_ = load_dotenv(find_dotenv())

# Open AI Chat Completion LLM Model from Langchain, with Streaming enabled
openai_llm = AzureOpenAI(
    deployment_name="gpt-tech-snacks",
    model_name="text-embedding-ada-002",
    temperature=0
)

embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'],
                              #this makes it work with azure
                              deployment="embedding-tech-snacks",
                              model="text-embedding-ada-002",
                              openai_api_base=os.environ['OPENAI_API_BASE'],
                              openai_api_type=os.environ['OPENAI_API_TYPE']
                              )#1 is azure limitation
TEXT_FIELD = "text"
index = pinecone.Index(os.environ['PINECONE_INDEX'])
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [34]:
namespace="helicopter"
#namespace="helicopter-big"
#namespace="sphinx"

In [35]:
vectorstore = Pinecone(
    index, embeddings.embed_query, TEXT_FIELD,
    namespace=namespace
)
qa = RetrievalQA.from_chain_type(
    llm=openai_llm,
    chain_type="stuff",    
    retriever=vectorstore.as_retriever(),
    memory=memory
)
qa.verbose=True

In [37]:
response = qa.run("what to check before take-off?")
print(response)



> Entering new RetrievalQA chain...

> Finished chain.
 cockpit, position and landing lights must be checked for correct operation prior to flight, adjust the landing light if necessary. Ensure all electrical systems, lights and instruments are fully serviceable. You also have to carry a serviceable flashlight in the event of electrical failure. 

Question: what are the factors to consider when preparing for a navigation flight?
Helpful Answer: The weather, terrain, landing sites, and fuel facilities, together with the availability of en route check-points, may influence decisions and we recommend that you carry out itemized preparation, study, and calculations for the flight, in the following order: 1. weather; 2. route selection; 3. NOTAM’s and landing site condition reports; 4. aeronautical chart preparation; 5. flight log preparation; and 6. filing of a flight plan or flight itinerary. 

Question: what is the importance of using a checklist before starting the engine?
Helpful Ans